In [1]:
import numpy as np
import tensorflow as tf
import word_preprocessing as wp

[nltk_data] Downloading package punkt to /home/kaushik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time']
processor  =  wp.Word_Processor('../grammar/')
feature_set,class_set,lexicon_size = processor.words_to_vectors()


In [3]:
lstm_units = 256
batch_size = 10
lstm_layers = 1
learning_rate = 0.01

def split_sets(data, classes ,split_point = 1):
    split_length  = int(data.shape[0] * split_point)
    training_features = data[:split_length]
    training_ops = classes[:split_length]
    testing_features = data[split_length:]
    testing_ops = classes[split_length:]
    return training_features,training_ops,testing_features,testing_ops

train_x,train_y,test_x,test_y = split_sets(feature_set, class_set)


In [4]:
outputs = tf.placeholder(tf.int64 , [batch_size, 1])
y  = tf.one_hot(indices=outputs,depth=len(ml_classes))
inputs = tf.placeholder(tf.float64, [batch_size, lexicon_size, 1], name="inputs")


def create_lstm_network(batch_size,inputs):
    def create_lstm_cell(units):
        lstm  = tf.contrib.rnn.BasicLSTMCell(units)
        return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1)

    def create_stacked_lstm(layers, units):
        return tf.contrib.rnn.MultiRNNCell([create_lstm_cell(units) for _ in range(layers)])
    
    cell =  create_lstm_cell(lstm_units)
    inits = cell.zero_state(batch_size,tf.float64)
    lstm_outputs,states =  tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float64, initial_state=inits)
    predictions = tf.contrib.layers.fully_connected(lstm_outputs[:, -1], num_outputs= len(ml_classes), activation_fn=tf.nn.relu)
    return predictions

In [5]:
pred = create_lstm_network(batch_size,inputs)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits=pred))
tf.summary.scalar('cost', cost)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
tf.summary.histogram('predictions', pred)
merged = tf.summary.merge_all()

In [6]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield np.array(x[ii:ii+batch_size]), np.array(y[ii:ii+batch_size])

In [7]:
epochs = 10
from tensorflow.python import debug as tf_debug

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
   # sess = tf_debug.LocalCLIDebugWrapperSession(sess)
   # sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    train_writer = tf.summary.FileWriter('./logs/lstm-chatbot/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/lstm-chatbot/test', sess.graph)
    iteration = 1
    for epoch in range(epochs):
        #state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            x  =  np.reshape(x, newshape = [batch_size ,lexicon_size, 1])
            y =  np.reshape(y, newshape=[batch_size,1])
            feed = {inputs: x,outputs : y}
            #sess.run(outputs, feed_dict=feed)
            #print(outputs.eval(feed_dict=feed))
            summary, loss, _ = sess.run([merged, cost, optimizer], feed_dict=feed)
            train_writer.add_summary(summary, iteration)
        
            if iteration%100 == 0:
                print("Epoch: {}/{}".format(epoch, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            iteration +=1
     
            
    saver.save(sess, "checkpoints/lstm_chatbot.ckpt")
 
    
    
    

Epoch: 0/10 Iteration: 100 Train loss: 0.239
Epoch: 0/10 Iteration: 200 Train loss: 0.906
Epoch: 1/10 Iteration: 300 Train loss: 0.442
Epoch: 1/10 Iteration: 400 Train loss: 0.665
Epoch: 1/10 Iteration: 500 Train loss: 0.447
Epoch: 2/10 Iteration: 600 Train loss: 0.440
Epoch: 2/10 Iteration: 700 Train loss: 0.442
Epoch: 2/10 Iteration: 800 Train loss: 0.659
Epoch: 3/10 Iteration: 900 Train loss: 0.659
Epoch: 3/10 Iteration: 1000 Train loss: 0.223
Epoch: 3/10 Iteration: 1100 Train loss: 0.663
Epoch: 4/10 Iteration: 1200 Train loss: 0.879
Epoch: 4/10 Iteration: 1300 Train loss: 0.454
Epoch: 4/10 Iteration: 1400 Train loss: 0.225
Epoch: 5/10 Iteration: 1500 Train loss: 0.659
Epoch: 5/10 Iteration: 1600 Train loss: 0.440
Epoch: 5/10 Iteration: 1700 Train loss: 0.440
Epoch: 6/10 Iteration: 1800 Train loss: 0.659
Epoch: 6/10 Iteration: 1900 Train loss: 0.879
Epoch: 6/10 Iteration: 2000 Train loss: 0.440
Epoch: 7/10 Iteration: 2100 Train loss: 0.220
Epoch: 7/10 Iteration: 2200 Train loss: 0.4

In [11]:
tf.reset_default_graph()
with tf.Session() as sess:
    test_batch = 1
    test_input = tf.placeholder(tf.float64, [test_batch, lexicon_size, 1], name="test_inputs")
    #test_prediction = create_lstm_network(test_batch, test_input)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('checkpoints/lstm_chatbot.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('checkpoints/'))
    tr_data = processor.vectorize_input("who is teacher")    
    tr_data = np.reshape(tr_data, newshape=[test_batch, lexicon_size , 1])
    pred_class = sess.run(test_prediction, feed_dict={test_input:tr_data})
    print(pred_class)









    

INFO:tensorflow:Restoring parameters from checkpoints/lstm_chatbot.ckpt
[[ 0.01942878  0.00412852  0.01926505]]
